<a href="https://colab.research.google.com/github/8MeTools/ButtonTextReColorizer/blob/main/ButtonTextReColorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Googleドライブのマウント**
Googleドライブにアクセスして、ファイルの読み書きなどを行うために必要なコードです。実行した後、アクセスを許可してください。\
なお、このノートブックでは指定されたディレクトリ以外のファイル(ご自身の写真やドキュメントデータなど)を読み書きすることはありませんので、ご安心ください。

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 環境の準備
以下のセルを実行すると、必要なツールの導入などを行います。

In [28]:
# 初回時のみ実行が必要です。2回目以降は実行する必要はありません。

# Googleドライブのマイドライブへ移動.
%cd /content/drive/MyDrive
#マイドライブ直下に8MeToolsフォルダを生成.
!mkdir -p ./8MeTools/BTRC
#8MeToolsフォルダに移動
%cd /content/drive/MyDrive/8MeTools
#8MeToolsフォルダ内に必要ファイルをダウンロード.
!git clone https://github.com/8MeTools/ButtonTextReColorizer.git
#フォルダ名のリネーム.
!mv "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"* "/content/drive/MyDrive/8MeTools/BTRC/"
#不要データの削除(GitHub上では必要ですが、作業においては必要のないものです)
!rm -r "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLYT/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/.gitkeep
# ファイル,ディレクトリが存在しているかどうかを確認する
%ls /content/drive/MyDrive/8MeTools/BTRC -l

/content/drive/MyDrive
/content/drive/MyDrive/8MeTools
Cloning into 'ButtonTextReColorizer'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 379 (delta 289), reused 361 (delta 279), pack-reused 0 (from 0)
Receiving objects: 100% (379/379), 182.59 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (289/289), done.
Updating files: 100% (538/538), done.


In [30]:
%cd /content
!git clone https://github.com/stblr/wuj5.git
!pip install json5
%cd /content/wuj5
!ls -l

/content
fatal: destination path 'wuj5' already exists and is not an empty directory.
/content/wuj5
total 92
-rw-r--r-- 1 root root 10175 Jun 25 09:19 bmg.py
-rw-r--r-- 1 root root  4875 Jun 25 09:19 brctr.py
-rw-r--r-- 1 root root 12666 Jun 25 09:19 brlan.py
-rw-r--r-- 1 root root 21882 Jun 25 09:19 brlyt.py
-rw-r--r-- 1 root root  7066 Jun 25 09:19 common.py
-rw-r--r-- 1 root root  1058 Jun 25 09:19 LICENSE
-rw-r--r-- 1 root root   755 Jun 25 09:19 README.md
-rw-r--r-- 1 root root  3645 Jun 25 09:19 u8.py
-rwxr-xr-x 1 root root  6782 Jun 25 09:19 wuj5.py
-rw-r--r-- 1 root root  3417 Jun 25 09:19 yaz.py


In [27]:
# これを実行したあと、"/content/wuj5"と出ていればOK
!pwd

/content/wuj5


# **カラーコードの定義**

In [31]:
import re

# カラーコード判定関数
def get_valid_hex_color(prompt):
    """ ユーザーにカラーコードを入力させ、正しい形式 (#RRGGBB) でなければ再入力を求める """
    hex_pattern = re.compile(r"^#([A-Fa-f0-9]{6})$")

    while True:
        color = input(prompt).strip()
        if hex_pattern.match(color):
            return color
        else:
            print("⚠ 無効なカラーコードです。#RRGGBB の形式で入力してください。")

# HTMLカラーコードをRGBに変換する関数
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# カラープリセット（デフォルト値）
color_presets = {
    "fuchi_pattern2": {"black": "#ffffff", "white": "#000000"},
    "color_base2": {"black": "#434343", "white": "#434343"},
    "black_base2": {"black": "#434343", "white": "#434343"},
    "pikapika": {"black": "#ffffff", "white": "#ffffff"},
    "color_yajirushi": {"black": "#ffffff", "white": "#000000"},
    "ability_graph2": {"black": "#434343", "white": "#434343"},
    "black_pt00": {"black": "#434343", "white": "#434343"},
    "black_pt01": {"black": "#434343", "white": "#434343"},
}

# カラー設定関数
def get_custom_color(name, default_black, default_white):
    print(f"\n=== {name} のカラー設定 ===")
    print(f"デフォルト → Black: {default_black}, White: {default_white}")

    # use_default = input("デフォルトのままにしますか？（Y/n）: ").strip().lower()
    use_default = "n"
    if use_default == "n":
        black = get_valid_hex_color(f"{name} BlackColor（縁取り色）を入力してください: ")
        white = get_valid_hex_color(f"{name} WhiteColor（文字色）を入力してください: ")
    else:
        black, white = default_black, default_white

    return hex_to_rgb(black), hex_to_rgb(white)

# ユーザーにカラーの変更を確認
fuchi_black_rgb, fuchi_white_rgb = get_custom_color("fuchi_pattern2", color_presets["fuchi_pattern2"]["black"], color_presets["fuchi_pattern2"]["white"])
base_black_rgb, base_white_rgb = get_custom_color("color_base2", color_presets["color_base2"]["black"], color_presets["color_base2"]["white"])
arrow_black_rgb, arrow_white_rgb = get_custom_color("color_yajirushi", color_presets["color_yajirushi"]["black"], color_presets["color_yajirushi"]["white"])
ability_black_rgb, ability_white_rgb = get_custom_color("ability_graph2", color_presets["ability_graph2"]["black"], color_presets["ability_graph2"]["white"])

print("\nカラー設定完了が完了しました。次のセルを実行してください。")



=== fuchi_pattern2 のカラー設定 ===
デフォルト → Black: #ffffff, White: #000000
fuchi_pattern2 BlackColor（縁取り色）を入力してください: #7d47fc
fuchi_pattern2 WhiteColor（文字色）を入力してください: #38148c

=== color_base2 のカラー設定 ===
デフォルト → Black: #434343, White: #434343
color_base2 BlackColor（縁取り色）を入力してください: #35245c
color_base2 WhiteColor（文字色）を入力してください: #35245c

=== color_yajirushi のカラー設定 ===
デフォルト → Black: #ffffff, White: #000000
color_yajirushi BlackColor（縁取り色）を入力してください: #ffffff
color_yajirushi WhiteColor（文字色）を入力してください: #7d47fc

=== ability_graph2 のカラー設定 ===
デフォルト → Black: #434343, White: #434343
ability_graph2 BlackColor（縁取り色）を入力してください: #7d47fc
ability_graph2 WhiteColor（文字色）を入力してください: #7d47fc

カラー設定完了が完了しました。次のセルを実行してください。


In [32]:
# テキストカラーの定義(縁取りカラーのみ変更可)
import re

def get_outline_color_from_user(state_name, default_hex):
    pattern = re.compile(r"^#([A-Fa-f0-9]{6})$")
    while True:
        user_input = input(f"{state_name} の縁取りカラーコード（例: #00FFAA）を入力（空欄でデフォルト: {default_hex}）: ").strip()
        if user_input == "":
            user_input = default_hex
        if pattern.match(user_input):
            rgb = tuple(int(user_input[i:i+2], 16) for i in (1, 3, 5))
            print(f"→ {state_name} outline color: {rgb}")
            return rgb
        else:
            print("⚠ 無効な形式です。#RRGGBB の形式で入力してください。")

# デフォルト文字色
default_free_text = (220, 220, 220)
default_select_text = (255, 255, 255)

# ユーザー入力：縁取り色のみ
free_outline = get_outline_color_from_user("free", "#787878")
select_outline = get_outline_color_from_user("select", "#FFFFFF")

# カラー定義（スクリプト本体で使われる変数）
text_free_colors = (default_free_text, free_outline)
text_select_colors = (default_select_text, select_outline)

# common_w103_wifi_menu.brlyt用のテキストカラー定義(それ以外はBRLANで変更)
text_black_rgb, text_white_rgb = (text_select_colors[1], text_select_colors[0])

free の縁取りカラーコード（例: #00FFAA）を入力（空欄でデフォルト: #787878）: #3f13a8
→ free outline color: (63, 19, 168)
select の縁取りカラーコード（例: #00FFAA）を入力（空欄でデフォルト: #FFFFFF）: #8552fa
→ select outline color: (133, 82, 250)


# **JSON5(レイアウトファイル)の編集**
画面左上のメニューバーから"**ランタイム**"を選択し、"**現在のセルとその下のセルを実行**"をクリックしてください。\
その後は自動でセルを実行してくれます。\
<img src="https://drive.google.com/uc?export=view&id=1jwwFOCjsb4ip9j8MqtgL2a3zUT1kjaS7" width = 50%>

In [33]:
# オリジナルファイルをtmpフォルダに移動
!cp -r "/content/drive/MyDrive/8MeTools/BTRC/Assets/BRLYT/"* "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/"

In [34]:
import json5
import os
import re

# JSON5ファイルが格納されているディレクトリ
json5_dir = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT"

# JSON5ファイルを再帰的に取得
json5_files = []
for root, _, files in os.walk(json5_dir):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

if not json5_files:
    print("指定されたディレクトリにJSON5ファイルが存在しません。")
else:
    for json5_file in json5_files:
        file_path = os.path.join(json5_dir, json5_file)

        with open(file_path, "r", encoding="utf-8") as file:
            json_text = file.read()

        # "name": 対象のブロックを検索
        pattern = re.compile(
            r'(\s*"name": "(fuchi_pattern2|color_base2|black_base2|pikapika|color_yajirushi|ability_graph2|black_pt00|black_pt01|text|chara02)".*?("tev color 1 a": \d+))',
            re.DOTALL
        )
        matches = list(re.finditer(pattern, json_text))

        if not matches:
            print(f"{json5_file} に編集対象のTexture Paneが含まれていないため、編集を行いません。")
            continue

        # JSONテキストを変更
        for match in reversed(matches):  # **後ろから置換**
            block_text = match.group(1)
            block_name = match.group(2)

            # 対応するカラーを取得
            color_map = {
                "fuchi_pattern2": (fuchi_black_rgb, fuchi_white_rgb),
                "color_base2": (base_black_rgb, base_white_rgb),
                "black_base2": (base_black_rgb, base_white_rgb),
                "pikapika": (base_black_rgb, base_white_rgb),
                "color_yajirushi": (arrow_black_rgb, arrow_white_rgb),
                "ability_graph2": (ability_black_rgb, ability_white_rgb),
                "black_pt00": (ability_black_rgb, ability_white_rgb),
                "black_pt01": (ability_black_rgb, ability_white_rgb),
                "text": (text_black_rgb, text_white_rgb),
                "chara02": (arrow_white_rgb, arrow_black_rgb), # ここはあえて逆にしている.矢印のカラー割り当てをオリジナルに準拠するため.
            }

            black_rgb, white_rgb = color_map.get(block_name, ((0, 0, 0), (255, 255, 255)))

            # "tev color 0" のRGB値を変更
            block_text = re.sub(r'"tev color 0 r": \d+(,?)', f'"tev color 0 r": {black_rgb[0]}\\1', block_text)
            block_text = re.sub(r'"tev color 0 g": \d+(,?)', f'"tev color 0 g": {black_rgb[1]}\\1', block_text)
            block_text = re.sub(r'"tev color 0 b": \d+(,?)', f'"tev color 0 b": {black_rgb[2]}\\1', block_text)

            # "tev color 1" のRGB値を変更
            block_text = re.sub(r'"tev color 1 r": \d+(,?)', f'"tev color 1 r": {white_rgb[0]}\\1', block_text)
            block_text = re.sub(r'"tev color 1 g": \d+(,?)', f'"tev color 1 g": {white_rgb[1]}\\1', block_text)
            block_text = re.sub(r'"tev color 1 b": \d+(,?)', f'"tev color 1 b": {white_rgb[2]}\\1', block_text)

            # **変更を適用**
            json_text = json_text[:match.start()] + block_text + json_text[match.end():]

        # JSON5ファイルを上書き保存
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(json_text)

        print(f"{json5_file} が変更されました。")


/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w004_menu.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w010_cup.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w016_yajirushi_left.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w019_mii_wifi.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w021_yajirushi_right.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w042_id.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w044_id_key.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w083_earth_chat_message.brlyt.json5 が変更されました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w084_earth_delete.brly

### JSON5をBRLYTファイルに変換するコード

In [35]:
import os
import subprocess

# JSON5ファイルが格納されている親フォルダ
folder_path = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT"

# フォルダ内のすべてのJSON5ファイルを再帰的に取得
json5_files = []
for root, _, files in os.walk(folder_path):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

if not json5_files:
    print("JSON5フォルダ内に処理対象のファイルがありません。")
else:
    print("処理対象のファイル:")
    for file_path in json5_files:
        print(file_path)

    for file_path in json5_files:
        # wuj5.py を実行
        command = ["python", "wuj5.py", "encode", file_path]
        print(f"実行中: {' '.join(command)}")

        try:
            result = subprocess.run(command, capture_output=True, text=True)
            print(f"出力: {result.stdout}")
            if result.stderr:
                print(f"エラー: {result.stderr}")
        except Exception as e:
            print(f"エラー発生: {e}")

    print("\nすべてのJSON5ファイルをエンコードしました！")

処理対象のファイル:
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w004_menu.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w010_cup.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w016_yajirushi_left.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w019_mii_wifi.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w021_yajirushi_right.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w042_id.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w044_id_key.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w083_earth_chat_message.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w084_earth_delete.brlyt.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/butt

In [36]:
#json5を削除
import os

# JSON5ファイルが格納されている親フォルダ
folder_path = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT"

# フォルダ内のすべてのJSON5ファイルを再帰的に取得
json5_files = []
for root, _, files in os.walk(folder_path):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

if not json5_files:
    print("削除対象のJSON5ファイルがありません。")
else:
    for json5_file in json5_files:
        file_path = os.path.join(folder_path, json5_file)
        os.remove(file_path)
        print(f"削除完了: {json5_file}")
    print("すべてのJSON5ファイルを削除しました。")

削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w004_menu.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w010_cup.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w016_yajirushi_left.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w019_mii_wifi.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w021_yajirushi_right.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w042_id.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w044_id_key.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w083_earth_chat_message.brlyt.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/Channel.d/button/blyt/common_w084_earth_delete.brlyt.json5
削除完了: /content/dri

In [37]:
#一時フォルダから完成フォルダに移動(BRLYT)
!mv "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/"* "/content/drive/MyDrive/8MeTools/BTRC/EditedBRLYT/"
print("BRLYTファイルへの変換が全て完了しました。Googleドライブから確認してください。")

BRLYTファイルへの変換が全て完了しました。Googleドライブから確認してください。


### BRLAN編


In [38]:
# JSON5ファイルをtmpフォルダに移動
!cp -r "/content/drive/MyDrive/8MeTools/BTRC/Assets/BRLAN/"* "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/"

In [39]:
import json5
import os

json5_dir = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN"
json5_files = []
for root, _, files in os.walk(json5_dir):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

# カラー更新関数
def update_tev_colors(data, start_outline, start_text, end_outline, end_text):
    for section in data.get("sections", []):
        for content in section.get("contents", []):
            if content.get("name") != "text":
                continue
            for animation in content.get("animations", []):
                for target in animation.get("targets", []):
                    kind = target.get("kind", "")
                    if not kind.startswith("tev color"):
                        continue

                    color_type = kind.split()[2]  # '0' or '1'
                    component = kind.split()[-1]  # 'r', 'g', 'b'

                    for key in target.get("keys", []):
                        # 遷移なし（同じ色で固定）
                        if start_outline == end_outline and start_text == end_text:
                            color = start_text if color_type == "0" else start_outline
                        else:
                            # 遷移あり（frame に応じて始点か終点の色を選ぶ）
                            color = start_text if (color_type == "0" and key["frame"] == 0.0) else \
                                    end_text if (color_type == "0" and key["frame"] != 0.0) else \
                                    start_outline if (color_type == "1" and key["frame"] == 0.0) else \
                                    end_outline

                        if component == "r":
                            key["value"] = color[0]
                        elif component == "g":
                            key["value"] = color[1]
                        elif component == "b":
                            key["value"] = color[2]
    return data

# 実行
if not json5_files:
    print("指定されたディレクトリにJSON5ファイルが存在しません。")
else:
    for json5_file in json5_files:
        with open(json5_file, "r", encoding="utf-8") as file:
            try:
                data = json5.load(file)
            except Exception as e:
                print(f"{json5_file}: 読み込みエラー - {e}")
                continue

        filename = os.path.basename(json5_file).lower()

        # ファイル名ルールによるカラー適用順
        if "free_to_select" in filename:
            start_outline, start_text = text_free_colors
            end_outline, end_text = text_select_colors
        elif "select_to_free" in filename:
            start_outline, start_text = text_select_colors
            end_outline, end_text = text_free_colors
        elif "free" in filename and "select" not in filename:
            start_outline = end_outline = text_free_colors[0]
            start_text = end_text = text_free_colors[1]
        elif "select" in filename and "free" not in filename:
            start_outline = end_outline = text_select_colors[0]
            start_text = end_text = text_select_colors[1]
        elif "stop" in filename:
            start_outline = end_outline = text_select_colors[0]
            start_text = end_text = text_select_colors[1]
        elif "common_w098_wifi_menu_text" in filename:
            start_outline = end_outline = text_select_colors[0]
            start_text = end_text = text_select_colors[1]
        else:
            print(f"{json5_file}: ファイル名から適用カラーが判断できません。スキップします。")
            continue

        # 更新
        updated_data = update_tev_colors(data, start_outline, start_text, end_outline, end_text)

        # 保存
        with open(json5_file, "w", encoding="utf-8") as file:
            json5.dump(updated_data, file, indent=2)

        print(f"{json5_file} の編集が完了しました。")


/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_fuchi_check_loop.brlan.json5: ファイル名から適用カラーが判断できません。スキップします。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_select.brlan.json5 の編集が完了しました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_stop.brlan.json5 の編集が完了しました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free.brlan.json5 の編集が完了しました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free_to_select.brlan.json5 の編集が完了しました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_fuchi_check_loop.brlan.json5: ファイル名から適用カラーが判断できません。スキップします。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select.brlan.json5 の編集が完了しました。
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select_to_

### JSON5をBRLANに変換するコード

In [40]:
# json5をBRLANに変換
import os
import subprocess

# JSON5ファイルが格納されている親フォルダ
folder_path = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN"

# フォルダ内のすべてのJSON5ファイルを再帰的に取得
json5_files = []
for root, _, files in os.walk(folder_path):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

if not json5_files:
    print("JSON5フォルダ内に処理対象のファイルがありません。")
else:
    print("処理対象のファイル:")
    for file_path in json5_files:
        print(file_path)

    for file_path in json5_files:
        # wuj5.py を実行
        command = ["python", "wuj5.py", "encode", file_path]
        print(f"実行中: {' '.join(command)}")

        try:
            result = subprocess.run(command, capture_output=True, text=True)
            print(f"出力: {result.stdout}")
            if result.stderr:
                print(f"エラー: {result.stderr}")
        except Exception as e:
            print(f"エラー発生: {e}")

    print("\nすべてのJSON5ファイルをエンコードしました！")

処理対象のファイル:
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_fuchi_check_loop.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_select.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_stop.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free_to_select.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_fuchi_check_loop.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select_to_free.brlan.json5
/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w016_yajirushi_lef

In [41]:
#json5を削除
import os

# JSON5ファイルが格納されている親フォルダ
folder_path = "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN"

# フォルダ内のすべてのJSON5ファイルを再帰的に取得
json5_files = []
for root, _, files in os.walk(folder_path):
    for f in files:
        if f.endswith(".json5"):
            json5_files.append(os.path.join(root, f))

if not json5_files:
    print("削除対象のJSON5ファイルがありません。")
else:
    for json5_file in json5_files:
        file_path = os.path.join(folder_path, json5_file)
        os.remove(file_path)
        print(f"削除完了: {json5_file}")
    print("すべてのJSON5ファイルを削除しました。")

削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_fuchi_check_loop.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_select.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w004_menu_text_light_02_stop.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_free_to_select.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_fuchi_check_loop.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Channel.d/button/anim/common_w010_cup_select_to_free.brlan.json5
削除完了: /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/Chan

In [42]:
#一時フォルダから完成フォルダに移動
!mv "/content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/"* "/content/drive/MyDrive/8MeTools/BTRC/EditedBRLAN/"
print("BRLANファイルへの変換が全て完了しました。Googleドライブから確認してください。")

BRLANファイルへの変換が全て完了しました。Googleドライブから確認してください。


Googleドライブを開いてファイルをダウンロードしてください。お疲れ様でした。